In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as MAE
import matplotlib.pyplot as plt
import re

# CSV 파일에서 데이터 로드
file_path = '20240703_correlation_copy.csv'
df = pd.read_csv(file_path)

# 열 목록
columns = df.columns

# R^2 값을 저장할 행렬 초기화
r2_matrix = pd.DataFrame(index=columns, columns=columns)

# 각 열을 종속 변수로 지정
for dependent_var in columns:
    independent_vars = df.columns.difference([dependent_var])
    
    for var in independent_vars:
        # 결측치 제거
        valid_data = df[[var, dependent_var]].dropna()
        
        X = valid_data[[var]]
        y = valid_data[dependent_var]
        
        if len(valid_data) > 0:  # 유효한 데이터가 있는지 확인
            # 다항식 특징 변환기 생성 (차수 2로 설정)
            poly = PolynomialFeatures(degree=3)
            X_poly = poly.fit_transform(X)
            
            # 모델 훈련
            model = LinearRegression()
            model.fit(X_poly, y)
            
            # 예측
            y_pred = model.predict(X_poly)
            
            # R^2 계산
            r2 = r2_score(y, y_pred)
            r2_matrix.loc[var, dependent_var] = r2

        else:
            r2_matrix.loc[var, dependent_var] = np.nan  # 유효한 데이터가 없으면 NaN으로 설정

# r2_matrix를 CSV 파일로 저장
output_file_path = '20240715_cubic_r2.csv'  # 저장할 파일 경로
r2_matrix.to_csv(output_file_path)

# R^2 값이 0.9 이상인 모델 찾기
high_r2_models = r2_matrix[r2_matrix >= 0.90].stack().reset_index()
high_r2_models.columns = ['Independent_Var', 'Dependent_Var', 'R2']
